# LangChain 101: Finetuning - Lecture 2c

Model finetuning, also known as transfer learning, is a machine learning technique used to improve the performance of a pre-existing model on a specific task by training it further on new data related to that task.
Finetuning is commonly employed in scenarios where a pretrained model has learned useful representations of a general domain (in our case natural language) and is then adapted to perform well on a narrower, more specific task.

This notebook is dedicated to finetuning a Falcon model with QLoRA.

Learn more about LLMs and Decoding Strategies from [LangChain 101 course](https://github.com/IvanReznikov/DataVerse/tree/main/Courses/LangChain/Lecture2.%20Models)

In [1]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
import pprint


def getpreferredencoding(do_setlocale=True):
    return "UTF-8"


locale.getpreferredencoding = getpreferredencoding

In [2]:
!pip install -q -U bitsandbytes einops safetensors torch xformers datasets
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requiremen

In [3]:
"""
einops==0.6.1 bitsandbytes==0.41.1 transformers==4.32.0
safetensors==0.3.2 xformers==0.0.21 huggingface-hub==0.16.4
"""

'\neinops==0.6.1 bitsandbytes==0.41.1 transformers==4.32.0 \nsafetensors==0.3.2 xformers==0.0.21 huggingface-hub==0.16.4\n'

# Import

In [4]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
)
import transformers
import torch
from torch.utils.data import DataLoader, Dataset

# Load Quantized Model

In [5]:
"""
For arabic language one may choose the following option:
model = BloomForCausalLM.from_pretrained('Naseej/noon-7b')
tokenizer = BloomTokenizerFast.from_pretrained('Naseej/noon-7b')

data = load_dataset("ashhadulislam/arabic_medical_test", split="train")
"""

# Define the model ID for the sharded FALCON model by vilsonrodrigues
model_id = "vilsonrodrigues/falcon-7b-instruct-sharded"

# Configure BitsAndBytesConfig for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Initialize the tokenizer using the model ID and set the pad token to be the same as the end of sentence token
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# Initialize the pre-trained model using AutoModelForCausalLM
pretrained_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0},
    trust_remote_code=True,
)

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [6]:
from peft import prepare_model_for_kbit_training

pretrained_model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(pretrained_model)

In [7]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [8]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainable%: 0.13058363808693696


# Prepare Data

In [9]:
from datasets import load_dataset

"""
We'll be using a dataset of Mayo Clinic symptoms and diseases.
It contains information about 1,058 rows (symptoms) and related diseases.
The file weughts 626 kB
"""

data = load_dataset("celikmus/mayo_clinic_symptoms_and_diseases_v1", split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1058 [00:00<?, ? examples/s]

In [10]:
data

Dataset({
    features: ['text', 'label'],
    num_rows: 1058
})

In [11]:
tokenizer.pad_token = tokenizer.eos_token

train_dataset = data.map(
    lambda x: {
        "input_text": f"symptoms: {x['text']}; most likely explanation: {x['label']}"
    }
)

# Tokenize the datasets
train_encodings = tokenizer(
    train_dataset["input_text"],
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors="pt",
)

Map:   0%|          | 0/1058 [00:00<?, ? examples/s]

In [12]:
class TextDataset(Dataset):
    def __init__(self, encodings):
        """
        Initialize a custom dataset for text inputs and encodings.

        Parameters:
            encodings (dict): A dictionary containing the encoded inputs.
        """
        self.encodings = encodings

    def __getitem__(self, idx):
        """
        Get an item from the dataset by index.

        Parameters:
            idx (int): The index of the item to retrieve.

        Returns:
            dict: A dictionary containing the encoded input and labels.
        """
        # Create an item dictionary with tensors for each encoding key
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

        # Copy input_ids to labels for tasks like language modeling
        item["labels"] = item["input_ids"].clone()

        return item

    def __len__(self):
        """
        Get the length of the dataset.

        Returns:
            int: The number of items in the dataset.
        """
        return len(self.encodings["input_ids"])

In [13]:
# Convert the encodings to PyTorch datasets
train_dataset = TextDataset(train_encodings)

# Example Before Fine Tuning

In [14]:
request_text = """
In crowded places, I feel cold in the tips of my fingers, I sweat with dizziness.
What is happening?
"""
# hint: agoraphobia

In [15]:
encoding = tokenizer(request_text, return_tensors="pt").to("cuda:0")
pretrained_model_output = pretrained_model.generate(
    input_ids=encoding.input_ids,
    attention_mask=encoding.attention_mask,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.25,
    eos_token_id=tokenizer.eos_token_id,
)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [16]:
pprint.pprint(tokenizer.decode(pretrained_model_output[0], skip_special_tokens=True))

('\n'
 'In crowded places, I feel cold in the tips of my fingers, I sweat with '
 'dizziness. \n'
 'What is happening?\n'
 "I'm sorry, I cannot provide a response as the prompt is incomplete and does "
 'not provide enough context for me to understand the situation.')


In [17]:
request_text = "I started to feel swelling  and itching around the mouth and throat after a salad with peanuts, cherry tomatoes and cheese. What may be the reason?"

In [18]:
encoding = tokenizer(request_text, return_tensors="pt").to("cuda:0")
pretrained_model_output = pretrained_model.generate(
    input_ids=encoding.input_ids,
    attention_mask=encoding.attention_mask,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.25,
    eos_token_id=tokenizer.eos_token_id,
)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [19]:
pprint.pprint(tokenizer.decode(pretrained_model_output[0], skip_special_tokens=True))

('I started to feel swelling  and itching around the mouth and throat after a '
 'salad with peanuts, cherry tomatoes and cheese. What may be the reason?\n'
 'It is possible that the peanuts, cherry tomatoes, and cheese may have caused '
 'an allergic reaction. It is important to seek medical attention if you '
 'experience severe symptoms such as difficulty breathing, swelling of the '
 'face, or hives.')


# Training

The Trainer object, which orchestrates the training process requires the `model` to be fine-tuned, the `train_dataset`, and various training configurations specified within the TrainingArguments class. These configurations include the number of training epochs, batch size, gradient accumulation, learning rate, optimization algorithm (optim), and more. The `data_collator` specifies how data is collated during training.

In [20]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    # eval_dataset=val_dataset,
    args=transformers.TrainingArguments(
        num_train_epochs=10,
        per_device_train_batch_size=8,
        gradient_accumulation_steps=4,
        warmup_ratio=0.05,
        max_steps=40,
        learning_rate=2.5e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        lr_scheduler_type="cosine",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [21]:
trainer.train()

<ipython-input-12-0f0c123b8ad6>:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.176800
2,2.164900
3,2.089600
4,2.111200
5,2.012200
6,2.109100
7,2.035700
8,2.027200
9,1.965700
10,2.046700


TrainOutput(global_step=40, training_loss=1.8507223963737487, metrics={'train_runtime': 967.3144, 'train_samples_per_second': 1.323, 'train_steps_per_second': 0.041, 'total_flos': 6492863730941952.0, 'train_loss': 1.8507223963737487, 'epoch': 1.2})

# Example After Fine Tuning

In [22]:
# Save model:
trained_model = (
    trainer.model.module if hasattr(trainer.model, "module") else trainer.model
)  # Take care of distributed/parallel training
trained_model.save_pretrained("outputs")

In [23]:
# Now we can inference our model:
lora_config = LoraConfig.from_pretrained("outputs")
loaded_model = get_peft_model(
    prepare_model_for_kbit_training(pretrained_model), lora_config
)

In [24]:
loaded_model.config.use_cache = True
loaded_model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): FalconForCausalLM(
      (transformer): FalconModel(
        (word_embeddings): Embedding(65024, 4544)
        (h): ModuleList(
          (0-31): 32 x FalconDecoderLayer(
            (self_attention): FalconAttention(
              (maybe_rotary): FalconRotaryEmbedding()
              (query_key_value): Linear4bit(
                in_features=4544, out_features=4672, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4544, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4672, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (dense): Linear4bit(in_

In [25]:
# Empty VRAM
del model
del trained_model
del trainer
import gc

gc.collect()

63

In [26]:
model_id = (
    "vilsonrodrigues/falcon-7b-instruct-sharded"  # sharded model by vilsonrodrigues
)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

pretrained_model = AutoModelForCausalLM.from_pretrained(
    model_id, quantization_config=bnb_config, device_map={"": 0}, trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [27]:
def generate_answer(query):
    """
    Generate responses from both the original model and PEFT model and compare their answers.

    Parameters:
        query (str): The user query for which responses are generated.

    Returns:
        None
    """
    # System and user prompts
    system_prompt = """Answer the following question truthfully.
          If you don't know the answer or the question is too complex,
          respond 'Kindly, consult a doctor for further queries.'."""
    user_prompt = f"""<HUMAN>: {query}
      <ASSISTANT>: """
    final_prompt = system_prompt + "\n" + user_prompt

    # Device and dashline
    device = "cuda:0"
    dashline = "-" * 50

    # Encode prompt and generate response from the original model
    encoding = tokenizer(final_prompt, return_tensors="pt").to(device)
    output = pretrained_model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.25,
        repetition_penalty=1.3,
        eos_token_id=tokenizer.eos_token_id,
    )
    text_output = tokenizer.decode(output[0], skip_special_tokens=True)

    # Print original model response
    pprint.pprint(dashline)
    pprint.pprint(f"ORIGINAL MODEL RESPONSE:\n{text_output}")
    pprint.pprint(dashline)

    # Encode prompt and generate response from the PEFT model
    peft_encoding = tokenizer(final_prompt, return_tensors="pt").to(device)
    peft_output = loaded_model.generate(
        input_ids=peft_encoding.input_ids,
        attention_mask=peft_encoding.attention_mask,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.25,
        repetition_penalty=1.3,
        eos_token_id=tokenizer.eos_token_id,
    )
    peft_text_output = tokenizer.decode(peft_output[0], skip_special_tokens=True)

    # Print PEFT model response
    pprint.pprint(f"PEFT MODEL RESPONSE:\n{peft_text_output}")
    pprint.pprint(dashline)

In [28]:
query = """In crowded places, I feel cold in the tips of my fingers, I sweat with dizziness. What may be happening?"""
generate_answer(query)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


'--------------------------------------------------'
('ORIGINAL MODEL RESPONSE:\n'
 'Answer the following question truthfully.\n'
 "          If you don't know the answer or the question is too complex, \n"
 "          respond 'Kindly, consult a doctor for further queries.'.\n"
 '<HUMAN>: In crowded places, I feel cold in the tips of my fingers, I sweat '
 'with dizziness. What may be happening?\n'
 '      <ASSISTANT>: <UNKNOWN>')
'--------------------------------------------------'
('PEFT MODEL RESPONSE:\n'
 'Answer the following question truthfully.\n'
 "          If you don't know the answer or the question is too complex, \n"
 "          respond 'Kindly, consult a doctor for further queries.'.\n"
 '<HUMAN>: In crowded places, I feel cold in the tips of my fingers, I sweat '
 'with dizziness. What may be happening?\n'
 '      <ASSISTANT>: <HUMAN> may be experiencing symptoms of hypothermia, '
 'which is a decrease in body temperature below normal. It is important to '
 'seek medical

In [29]:
query = """I started to feel swelling  and itching around the mouth and throat after a salad with peanuts, cherry tomatoes and cheese. What may be the reason?"""
generate_answer(query)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


'--------------------------------------------------'
('ORIGINAL MODEL RESPONSE:\n'
 'Answer the following question truthfully.\n'
 "          If you don't know the answer or the question is too complex, \n"
 "          respond 'Kindly, consult a doctor for further queries.'.\n"
 '<HUMAN>: I started to feel swelling  and itching around the mouth and throat '
 'after a salad with peanuts, cherry tomatoes and cheese. What may be the '
 'reason?\n'
 "      <ASSISTANT>: 'It is possible that you may be allergic to peanuts. "
 "Kindly consult a doctor for further queries.'")
'--------------------------------------------------'
('PEFT MODEL RESPONSE:\n'
 'Answer the following question truthfully.\n'
 "          If you don't know the answer or the question is too complex, \n"
 "          respond 'Kindly, consult a doctor for further queries.'.\n"
 '<HUMAN>: I started to feel swelling  and itching around the mouth and throat '
 'after a salad with peanuts, cherry tomatoes and cheese. What may be